In [1]:
import time
import pandas as pd
import numpy as np
import os
import glob
import sys

from scipy.optimize import Bounds
import matplotlib.pyplot as plt
from matplotlib.widgets import TextBox

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#sys.path.append("/content/drive/MyDrive/ABAI/ABAI_project_CRMP")


In [3]:
#подгружаем CRMP
from liquid_models.crmp import CRMP

In [4]:
#подгружаем все ансамбли
from oil_models.ensemble_1 import LinearAndExpEnsemble
from oil_models.ensemble_2 import LinearExpEnsembleOnInjection

In [5]:
#определяем путь к папке с результатами
result_path = "results"

## Данные

In [6]:
# production = pd.read_excel('/content/drive/MyDrive/ABAI/ABAI_project_CRMP/2021.06.18_dataset_База_данных(01.01.2017-01.04.2021).xlsx', sheet_name = 'База добычи')
# GTM = pd.read_excel('/content/drive/MyDrive/ABAI/ABAI_project_CRMP/2021.06.18_dataset_База_данных(01.01.2017-01.04.2021).xlsx', sheet_name = 'ГТМ')
# injection = pd.read_excel("/content/drive/MyDrive/ABAI/ABAI_project_CRMP/2021.06.18_dataset_База_данных(01.01.2017-01.04.2021).xlsx", sheet_name='База закачки')
production = pd.read_excel('/content/drive/MyDrive/ABAI/ABAI_project_CRMP/2021.06.18_dataset_База_данных(01.01.2017-01.04.2021).xlsx', sheet_name = 'База добычи')
GTM = pd.read_excel('/content/drive/MyDrive/ABAI/ABAI_project_CRMP/2021.06.18_dataset_База_данных(01.01.2017-01.04.2021).xlsx', sheet_name = 'ГТМ')
injection = pd.read_excel("/content/drive/MyDrive/ABAI/ABAI_project_CRMP/2021.06.18_dataset_База_данных(01.01.2017-01.04.2021).xlsx", sheet_name='База закачки')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ABAI/ABAI_project_CRMP/2021.06.18_dataset_База_данных(01.01.2017-01.04.2021).xlsx'

In [15]:
clusters = {
     'cluster1': {
         'names_wells': ['AKSH_' + x for x in "223, 226, 246, 298, 430, 458, 507".split(', ')],
         'injwell': 'AKSH_214'
     },
     'cluster2': {
         'names_wells': ['AKSH_' + x for x in "236, 421, 426, 435, 446, 462".split(', ')],
         'injwell': 'AKSH_221'
     },
     'cluster3': {
         'names_wells': ['AKSH_' + x for x in "428".split(', ')],
         'injwell': 'AKSH_225'
     },
    'cluster4': {
         'names_wells': ['AKSH_' + x for x in "211, 224, 437, 463, 509, 516".split(', ')],
         'injwell': 'AKSH_240'
     },
     'cluster5': {
         'names_wells': ['AKSH_' + x for x in "203, 427, 464, 503, 512".split(', ')],
         'injwell': 'AKSH_247'
     },
    'cluster6': {
        'names_wells': ['AKSH_' + x for x in "222, 425, 461, 504, 515, 517".split(', ')],
        'injwell': 'AKSH_424'
    },
    'cluster7': {
        'names_wells': ['AKSH_' + x for x in "212, 505".split(', ')],
        'injwell': 'AKSH_249'
    },
        'cluster8': {
        'names_wells': ['AKSH_' + x for x in "306".split(', ')],
        'injwell': 'AKSH_220'
    },
        'cluster9': {
        'names_wells': ['AKSH_' + x for x in "218, 244, 251, 344, 416, 419, 438, 441, 466, 478, 502, 506, 514".split(', ')],
        'injwell': 'AKSH_217'
    },
        'cluster10': {
        'names_wells': ['AKSH_' + x for x in "213, 229".split(', ')],
        'injwell': 'AKSH_241'
    },
        'cluster11': {
        'names_wells': ['AKSH_' + x for x in "237, 238, 289, 508, 511".split(', ')],
        'injwell': 'AKSH_250'
    },
        'cluster12': {
        'names_wells': ['AKSH_' + x for x in "215, 216, 254, 292".split(', ')],
        'injwell': 'AKSH_299'
    }
}

In [16]:
#injection[injection['wellname'] == 'AKSH_299'].reset_index(drop=True).copy()
#production[production['wellname'] == 'AKSH_299'].reset_index(drop=True).columns

In [17]:
def crmp_plot_and_save(real, pred, well_name, cluster, test_size):
    fig = plt.figure(figsize=(20,10))
    plt.plot([x for x in range(real.shape[0])], real, label='real_values')
    plt.plot([x for x in range(pred.shape[0])], pred, label='predictions')

    plt.grid()
    plt.title(f"Values for well {well_name} in {cluster}")
    plt.legend(fontsize='xx-large')

    #plt.show()
    fig.savefig(f'results/Liq - {well_name} in {cluster}.jpg')
    print(f'Result for {well_name} saved!')

def oil_plot_and_save(real, pred, well_name, cluster, test_size, ens_number):
    fig = plt.figure(figsize=(20,10))
    ax1 = fig.add_axes([0.1, 0.2, 0.8, 0.7])
    ax2 = fig.add_axes([0.1, 0.05, 0.8, 0.075])
    ax1.plot([x for x in range(real.shape[0])], real, label='real_values')
    ax1.plot([x for x in range(pred.shape[0])], pred, label='predictions')

    error_abs = np.mean(np.abs(real[-test_size:] - pred[-test_size:]) / real[-test_size:]) * 100
    error_ave = np.mean((real[-test_size:] - pred[-test_size:]) / real[-test_size:]) * 100

    ax1.grid()
    ax1.set_title(f"Values for well {well_name} in {cluster}")
    ax1.legend(fontsize='xx-large')

    text = TextBox(ax2, label='Errors', initial=f"Error abs = {error_abs}" + ", " + f"Error ave = {error_ave}")  
    ax1.relim()
    ax2.relim()
    #plt.show()
    #os.mkdir(f'results/{cluster}')
    fig.savefig(f'results/{cluster}_Ensemble_{ens_number}_Oil - {well_name} in {cluster}.jpg')
    print(f'Result for {well_name} saved!')
    return error_abs, error_ave

def clean_path(path):
    files = glob.glob(path)
    for f in files:
        os.remove(f)

def train_test_split_data(lib_data, data_slice, test_size):
    train = {}
    test = {}
    all_data = {}
    for key, elem in lib_data.items():
        train[key] = elem[:-test_size][-data_slice:]
        test[key] = elem[-test_size:]
        all_data[key] = elem[- (test_size + data_slice):]
    return train, test, all_data


In [22]:
final_data = pd.DataFrame(columns=['well_name', 'date', 'real_oil', '1_ensemble', '2_ensemble'])
final_coef = pd.DataFrame(columns=['well_name','1_ensemble_coef_linear', '1_ensemble_coef_exp', 
                                    '2_ensemble_coef_linear', '2_ensemble_coef_exp', '2_ensemble_coef_linear_injection', 'error_abs_1',
                              'error_ave_1',
                              'error_abs_2',
                              'error_ave_2'])
for cluster in clusters:
    print('-'*70)
    print(cluster)
    print('-'*70)
    injwell = clusters[cluster]['injwell']
    for well_name in clusters[cluster]['names_wells']:
        print('Результаты для скважины - ', well_name)
        well_df = production[production['wellname'] == well_name].reset_index(drop=True).copy()

        mask_date = well_df['start_datetime'].duplicated()
        

        well_df = well_df[mask_date == False]

        a = well_df[['start_datetime', 'liq_vol, m3']]
        b = injection[['start_datetime', 'water_inj_vol']][injection['wellname'] == injwell]
        b['start_datetime'] = pd.to_datetime(injection['start_datetime'][injection['wellname'] == injwell])
        c = pd.merge(a,b, on='start_datetime', how='left').fillna(0)
        Inj_well = c['water_inj_vol'].values


        #подготавливаем все данные для данной скважины
        #ВАЖНО: класс CRMP ничего не знает о данных, поэтому важно осуществить проверку на совпадение по размерам и наличию NaNs
        #mask = well_df['liq_vol, m3'].values > 0
        
        # date_full = set(well_df['start_datetime'].values)
        
        # q0_well = well_df['liq_vol, m3'][mask_date == False][pd.to_datetime(well_df['start_datetime']).isin(list(date_full))].values
        # Pwf_well = well_df['bhp_final, bar (fact)'][mask_date == False][pd.to_datetime(well_df['start_datetime']).isin(list(date_full))].values
        # date_well = well_df['start_datetime'][mask_date == False][pd.to_datetime(well_df['start_datetime']).isin(list(date_full))].values
        # Inj_well = Inj['water_inj_vol'][pd.to_datetime(Inj['start_datetime']).isin(list(date_full))].values
        # qoil_well = well_df['oil_vol, m3'][mask_date == False][pd.to_datetime(well_df['start_datetime']).isin(list(date_full))].values
        q0_well = well_df['liq_vol, m3'].values
        Pwf_well = well_df['bhp_final, bar (fact)'].values
        date_well = well_df['start_datetime'].values
        qoil_well = well_df['oil_vol, m3'].values


        # print(len(date_full))
        # print(Pwf_well.shape)
        # print(q0_well.shape)
        # print(date_well.shape)
        # print(Inj_well.shape)
        # print(qoil_well.shape)
        
        mask = q0_well > 0
        q0_well = q0_well[mask]
        Pwf_well = Pwf_well[mask]
        date_well = date_well[mask]
        Inj_well = Inj_well[mask]
        qoil_well = qoil_well[mask]
        
        #проверка на равенство всех размеров
        assert Inj_well.shape == Pwf_well.shape
        assert Inj_well.shape == q0_well.shape
        assert q0_well.shape == date_well.shape
        assert qoil_well.shape == date_well.shape

        #устанавливаем, сколько данных берем с конца
        #можно сделать в процентах
        test_size = 30
        data_slice = 100

        if qoil_well.shape[0] < data_slice:
            print("Error! Small shape of data!")
            print('-'*70)
            continue

        train, test, all_data = train_test_split_data({'pwf': Pwf_well, 
                                            'q0': q0_well, 
                                            'date': date_well, 
                                            'inj': Inj_well, 
                                            'oil': qoil_well}, 
                                            data_slice=data_slice, 
                                            test_size=test_size)
        

        print(train['pwf'].shape)
        print(test['pwf'].shape)

        #создаем CRMP с базовыми параметрами
        train_model = CRMP(Vp=800000, 
                    PI=0.3, 
                    Coef_1=0.5, 
                    Date=train['date'], 
                    Inj=train['inj'], 
                    q0=train['q0'], 
                    Pwf=train['pwf'], 
                    two_inj=False)
        bounds = Bounds([0.5, 0.1, 0.01], [25, 7, 1.0])
        x0 = np.array([0.1, 0.5, 0.2])

        #запускаем поиск оптимальных параметров и предсказываем жидкость
        liq_prediction_for_train = train_model.minimize_and_predict(bounds, x0)

        parameters = train_model.parameters
        print(train_model.parameters)


        final_model = CRMP(Vp=parameters['Vp'], 
                    PI=parameters['PI'], 
                    Coef_1=parameters['Coef_1'], 
                    Date=all_data['date'], 
                    Inj=all_data['inj'], 
                    q0=all_data['q0'], 
                    Pwf=all_data['pwf'], 
                    two_inj=False)

        liq_prediction_final = np.array(list(all_data['q0'][:-test_size]) + list(final_model.predict().reshape((-1,))[-test_size:]))

        print(liq_prediction_final.shape)
        print(all_data['q0'].shape)
    
        #отрисовываем график жидкости и сохраняем
        #crmp_plot_and_save(all_data['q0'], liq_prediction_final, well_name, cluster, test_size)
        print('-'*70)

        # перед применением ансамбля жидкость берем предсказанную
        train['q0'] = liq_prediction_final[:-test_size]
        test['q0'] = liq_prediction_final[-test_size:]
        all_data['q0'] = liq_prediction_final

        # фильтруем данные от выбросов ВНФ
        f = np.array(((all_data['q0'] - all_data['oil']) / (all_data['oil'] + 1e-6)))
        #print(np.percentile(f, 97))
        mask_VNF = np.abs(f) < np.percentile(f, 97)
        all_data['q0'] = all_data['q0'][mask_VNF] 
        all_data['oil'] = all_data['oil'][mask_VNF]
        all_data['inj'] = all_data['inj'][mask_VNF]
        all_data['date'] = all_data['date'][mask_VNF]

        train['q0'] = all_data['q0'][:-test_size]
        test['q0'] = all_data['q0'][-test_size:]
        train['oil'] = all_data['oil'][:-test_size]
        test['oil'] = all_data['oil'][-test_size:]
        train['inj'] = all_data['inj'][:-test_size]
        test['inj'] = all_data['inj'][-test_size:]

        # создаем ансамбль 1
        OIZ = 3000 #TODO: (брать значение из таблицы)
        ensemble = LinearAndExpEnsemble(oil_train=train['oil'], 
                                        liq_train=train['q0'], 
                                        liq_test=test['q0'],
                                        OIZ=OIZ)
        oil_prediction_final, coef1_1, coef2_1 = ensemble.predict()

        #err_abs_1, err_ave_1 = oil_plot_and_save(all_data['oil'], oil_prediction_final, well_name, cluster, test_size, 1)


        #меняем немного данные, отрезаем часть для поиска коэффиентов
        period_train = 50
        if period_train >= train['q0'].shape[0]:
            period_train = 1
        base = {}
        base['q0'] = train['q0'][:-period_train]
        train['q0'] = train['q0'][-period_train:]
        base['oil'] = train['oil'][:-period_train]
        train['oil'] = train['oil'][-period_train:]
        base['inj'] = train['inj'][:-period_train]
        train['inj'] = train['inj'][-period_train:]

        #создаем ансамбль 2
        OIZ = 3000
        ensemble_2 = LinearExpEnsembleOnInjection(oil=base['oil'], 
                                                  oil_train=train['oil'], 
                                                  liq=base['q0'], 
                                                  liq_train=train['q0'], 
                                                  liq_test=test['q0'], 
                                                  Inj=base['inj'], 
                                                  Inj_train=train['inj'], 
                                                  Inj_test=test['inj'], 
                                                  OIZ=OIZ)
        #fit and predict
        status = ensemble_2.fit()
        if status < 0:
            print('-'*70)
            continue
        oil_prediction_final_2, coef1_2, coef2_2, coef3_2 = ensemble_2.predict()

        #err_abs_2, err_ave_2 = oil_plot_and_save(all_data['oil'], oil_prediction_final_2, well_name, cluster, test_size, 2)


        print(all_data['oil'].shape[0])
        print(all_data['date'].shape[0])
        print(oil_prediction_final.shape)
        print(oil_prediction_final_2.shape)

        df_well = pd.DataFrame({'well_name': np.array([well_name for x in range(all_data['oil'].shape[0])]),
                                'date': all_data['date'],
                                'real_oil': all_data['oil'], 
                                '1_ensemble': oil_prediction_final, 
                                '2_ensemble': oil_prediction_final_2})
        final_data = pd.concat((final_data, df_well))
        # coefs = pd.DataFrame({'well_name': np.array([well_name]),
        #                       '1_ensemble_coef_linear': float(coef1_1),
        #                       '1_ensemble_coef_exp': float(coef2_1), 
        #                       '2_ensemble_coef_linear': float(coef1_2), 
        #                       '2_ensemble_coef_exp': float(coef2_2), 
        #                       '2_ensemble_coef_linear_injection': float(coef3_2),
        #                       'error_abs_1':err_abs_1,
        #                       'error_ave_1':err_ave_1,
        #                       'error_abs_2':err_abs_2,
        #                       'error_ave_2':err_ave_2})
        #final_coef = pd.concat((final_coef, coefs))

        #раскомменчиваем, если нужно больше одной скважины

# final_data.to_excel(f"results/output_prediction.xlsx", index=False, sheet_name='predictions') 
# final_coef.to_excel(f"results/output_prediction.xlsx", index=False, sheet_name='coefs') 

----------------------------------------------------------------------
cluster1
----------------------------------------------------------------------
Результаты для скважины -  AKSH_223
(100,)
(30,)
Start the minimize function....Please wait
Total time =  1.4222540855407715
{'Vp': 0.5, 'PI': 0.1, 'Coef_1': 0.5367446172140248, 'Coef_2': 0, 'Tau': 1325218933.3333335}
(130,)
(130,)
----------------------------------------------------------------------
100
alpha -  0.6483311999243083
beta -  0.0014417246334396403
coef linear -  0.9129441833051124
coef exp -  0.0870558166948876
Period train -  100  samples
coef linear -  1.6130221526012816e-17
coef exp -  0.07826194620935784
coef linear 2 -  0.9217380537906427
Period train -  78  samples
126
126
(126,)
(126,)
Результаты для скважины -  AKSH_226
(100,)
(30,)
Start the minimize function....Please wait
Total time =  0.6421709060668945
{'Vp': 0.5, 'PI': 0.1, 'Coef_1': 0.20000041073390906, 'Coef_2': 0, 'Tau': 1325218933.3333335}
(130,)
(130,)
-

In [ ]:
final_coef

,well_name,1_ensemble_coef_linear,1_ensemble_coef_exp,2_ensemble_coef_linear,2_ensemble_coef_exp,2_ensemble_coef_linear_injection,error_abs_1,error_ave_1,error_abs_2,error_ave_2
0,AKSH_223,0.912944,0.087056,1.613022e-17,0.078262,0.921738,43.221301,43.221301,14.632578,-1.208522


In [ ]:
print('done')

done


In [ ]:
#запустить эту строку, чтобы очистить папку с результатами
clean_path("results/*")

In [23]:
production

,completion,wellname,dome,start_datetime,Год,Месяц,wellname&start_datetime,"liq_vol, m3","bhp_fluid, bar","bhp_pump, bar",...,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41
0,AKSH_11:М-II-1,AKSH_11,Северный купол,2019-02-06,2019,2,AKSH_11_43502,1.50693,131.215,133.715,...,Разделение по категориям bhp с 01.01.2017-01.0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AKSH_11:М-II-1,AKSH_11,Северный купол,2019-02-07,2019,2,AKSH_11_43503,26.07957,131.197,133.698,...,№,Кол.строк,Описание категорий,Фильтр,2017.0,2018.0,2019.0,2020.0,2021.0,2017-2021
2,AKSH_11:М-II-1,AKSH_11,Северный купол,2019-02-08,2019,2,AKSH_11_43504,26.19168,54.739,55.679,...,1,50744,Только значения bhp_pump,"фильтр qж>0, фильтр bhp_pump>0",0.0,13227.0,15771.0,17268.0,4478.0,50744
3,AKSH_11:М-II-1,AKSH_11,Северный купол,2019-02-09,2019,2,AKSH_11_43505,26.34346,53.607,54.527,...,2,7581,"bhp_pump отсутствует, взять с bhp_fluid","фильтр qж>0, фильтр bhp_pump=<0, фильтр bhp_fl...",0.0,2247.0,2253.0,2537.0,544.0,7581
4,AKSH_11:М-II-1,AKSH_11,Северный купол,2019-02-10,2019,2,AKSH_11_43506,26.19861,54.549,55.489,...,3,13567,bhp_pump и bhp_fluid отсутствуют,"фильтр qж>0, фильтр bhp_pump=<0, фильтр bhp_fl...",12927.0,502.0,68.0,64.0,6.0,13567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73937,AKSH_517:М-II-1,AKSH_517,Южный купол,2021-03-27,2021,3,AKSH_517_44282,22.04112,38.471,39.092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73938,AKSH_517:М-II-1,AKSH_517,Южный купол,2021-03-28,2021,3,AKSH_517_44283,20.78581,38.471,39.092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73939,AKSH_517:М-II-1,AKSH_517,Южный купол,2021-03-29,2021,3,AKSH_517_44284,20.79487,38.471,39.092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73940,AKSH_517:М-II-1,AKSH_517,Южный купол,2021-03-30,2021,3,AKSH_517_44285,20.91672,38.471,39.092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
